In [1]:
import pygame
import pygame_gui
import sys
import numpy as np

pygame-ce 2.4.0 (SDL 2.28.5, Python 3.11.6)


In [2]:
initial_board = np.array([
    [ 4, 6, 7, 1, 2, 7, 6, 4 ],
    [ 8, 8, 8, 8, 8, 8, 8, 8 ],
    [ 0, 0, 0, 0, 0, 0, 0, 0 ],
    [ 0, 0, 0, 0, 0, 0, 0, 0 ],
    [ 0, 0, 0, 0, 0, 0, 0, 0 ],
    [ 0, 0, 0, 0, 0, 0, 0, 0 ],
    [ 18, 18, 18, 18, 18, 18, 18, 18 ],
    [ 14, 16, 17, 11, 12, 17, 16, 14 ],
])

In [3]:
# piece id to sprite id
def pid2sid(id):
    assert id in range(1, 21), "Piece ID must be between 0 and 20."
    if id > 10:
        id -= 10
        i = 6
    else:
        i = 0

    if id == 1:
        j = 0
    elif id == 2 or id == 3:
        j = 1
    elif id == 4 or id == 5:
        j = 2
    elif id == 6:
        j = 3
    elif id == 7:
        j = 4
    elif id == 8 or id == 9 or id == 10:
        j = 5
    else:
        assert False, "Logic error."

    return i + j

In [4]:
class ChessGame():
    def __init__(self, board=None, current_player=1):
        self.board = initial_board.copy() if board is None else board
        self.current_player = current_player

    def copy(self):
        return ChessGame(board=self.board.copy(), current_player=self.current_player)
    
    @staticmethod
    def player_pid_range(player):
        if player == 1: # white
            return range(11, 21)
        elif player == 2:
            return range(1, 11)
        else:
            assert False, "1 and 2 are the only valid player ids."

    @staticmethod
    def in_bounds(pos):
        try:
            x, y = pos
            if not x in range(0, 8) or not y in range(0, 8):
                return False
        except:
            assert False, "pos is not a position."

        return True

    # assert pos is a pos
    # return pid at pos if pos contains one of current_player's live pieces
    # else return None
    def player_piece(self, pos):
        assert self.in_bounds(pos), "Position must be in bounds"
        
        x, y = pos
        if self.board[y][x] in self.player_pid_range(self.current_player):
            return self.board[y][x]
        else:
            return False

In [5]:
# Actual game/render loop
# -----------------------

# Initialize Pygame
pygame.init()

# Constants
WIDTH, HEIGHT = 400, 400
TILE_SIZE = WIDTH // 8

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

# Game data
game = ChessGame()
cursor = None

# Load sprite sheet
spritesheet = pygame.image.load('ChessPiecesArray.png')
sprite_size = spritesheet.get_size()[1]//2 # height / 2
sprites = [
    pygame.transform.scale(
        spritesheet.subsurface(pygame.Rect(j*sprite_size, i*sprite_size, sprite_size, sprite_size)), # Surface of (i, j)-th sprite
        (TILE_SIZE, TILE_SIZE) # size to resize to
    ) for i in range(2) for j in range(6)
]

# Initialize the screen
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Simple Chess")

# cursor col,row position to pixel location
def pos2pix(pos):
    (r, c) = pos
    return (TILE_SIZE*r, TILE_SIZE*c)

# Main game loop
running = True
while running:    
    # CONTROL
    # ------------------------------------------------------------    
    for event in pygame.event.get():
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
                continue
        elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:  # Left mouse button click
            x, y = event.pos
            ccol = x // TILE_SIZE
            crow = y // TILE_SIZE
            cursor = (ccol, crow)
            print(game.player_piece(cursor))


    # UPDATE
    # ------------------------------------------------------------

    # RENDER
    # ------------------------------------------------------------
    for row in range(8):
        for col in range(8):
            # tile colour
            color = (0xd1, 0xe3, 0xff) if (row + col) % 2 == 0 else (0x57, 0x97, 0xff)
            # cursor tile colour
            #if (col, row) == cursor and game.board[row][col] != 0:
            if (col, row) == cursor and game.player_piece(cursor):
                color = (0x57, 0xff, 0xbb)
            # draw tile
            pygame.draw.rect(screen, color, (col * TILE_SIZE, row * TILE_SIZE, TILE_SIZE, TILE_SIZE))
            # draw sprites
            if game.board[row][col] in range(1, 21):
                screen.blit(sprites[pid2sid(game.board[row][col])], pos2pix((col, row)))
    
    pygame.display.flip()


pygame.quit()
#sys.exit()        


18
12
False
